In [3]:
from nltk.tokenize.casual import casual_tokenize
import gensim
from gensim import corpora
import re
import pandas as pd
from string import punctuation
from gensim.models import CoherenceModel
import os

input_filepath="../../data/processed/"
filename="trisma2016_cleaned_text_by_lga.csv"

In [4]:


tweetdf=pd.read_csv(os.path.join(input_filepath, filename),header=0)
tweetdf

,Unnamed: 0,lga,text,cleaned
0,0,Albury,#2015bestnine🌟 #BestOfNine2015 #byebye #2015 #...,#2015bestnine #bestofnine2015 #byebye #2015 #h...
1,1,Armidale Regional,Tranquility in the Park. @ Moore Park Inn Bout...,tranquility in the park moore park inn boutiqu...
2,2,Ballina,Happy New Year to all my pals back home in the...,happy new year to all my pals back home in the...
3,3,Balranald,Crackin night on the river. https://t.co/lGYg4...,crackin night on the river just posted photo o...
4,4,Bathurst Regional,@ReneeYoungWWE Happy New Year to you &amp; you...,happy new year to you your fam as well miss mi...
5,5,Bayside,@tamasys @mattRan wtf is maths circus? @toddjt...,wtf is maths circus sounds like it must've bee...
6,6,Bega Valley,Suns out guns out. Coffee time. #crossfitgirls...,suns out guns out coffee time #crossfitgirls #...
7,7,Bellingen,More from the Promised Lands #normanheywood #l...,more from the promised lands #normanheywood #l...
8,8,Berrigan,@MoniqueAnnalyss hotttt 🔥🔥🔥 @MoniqueAnnalyss a...,hottt appreciate your hotness monique it's 201...
9,9,Blacktown,Ahhh Tumblr. I currently have a hammock in the...,ahhh tumblr currently have hammock in the livi...


In [5]:
#now split by various doc types and run LGA. DO i need to write out interim files? Not really, not yet

by_lga = tweetdf

In [6]:
#by_lga.apply(lambda x: len(x['text']), axis=1)

In [9]:
punc_regex = re.compile('[%s]' % re.escape(punctuation))

#finds best ntopics
def run_lda(text_series, start, step, stop):
    toks = [x.split(" ") for x in text_series] # join into a 2d list
    
    dictionary = corpora.Dictionary(toks)

    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in toks]

    Lda = gensim.models.ldamodel.LdaModel

    # Running and Trainign LDA model on the document term matrix.
    #ldamodel = Lda(doc_term_matrix, num_topics=num_topics, id2word = dictionary, passes=50)
    #return ldamodel
    model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=doc_term_matrix, texts=toks, start=start, limit=stop, step=step)
    return model_list, coherence_values



In [7]:
## reference: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#17howtofindtheoptimalnumberoftopicsforlda

def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus, num_topics=num_topics, id2word = dictionary, passes=50)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
start, step, stop = (100, 50, 500)
ldams, coh = run_lda(by_lga['text'], start, step, stop)



/Users/tdm/git/capstone/venv/lib/python3.6/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
x = range(start, stop, step)
plt.plot(x, coh)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()